In [1]:
#Omero
from myconfig import OMEROUSER, OMEROPASS, OMEROPORT, OMEROHOST
import ezomero
import omero

#Other
import numpy as np
import pandas as pd
import skimage.io
import os
import pathlib
import skimage
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns
from datetime import datetime
import warnings
import time
import glob
import PIL
import cv2
import csv



#Settings
warnings.filterwarnings('ignore')
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
# Variables
plate_id = 765
plate_name = "LM2_siGENOME_1A"
filepath = r"Z:\Data\@MiN\Sarah\Projects\TiM2023\ExampleData\Results_All"
filename = r"Z:\Data\@MiN\Sarah\Projects\TiM2023\ExampleData\Results_All\Image.csv"
index_file = f"Z:\Data\@MiN\Sarah\Projects\TiM2023\ExampleData\ImageIndex.ColumbusIDX.csv"

In [3]:
df_features = pd.read_csv(filename, delimiter=",")

In [ ]:
for x,y in enumerate(df_features.columns):
    print(f"{x}: {y}")

In [4]:
select = [5, 7, 27, 50, 58, 59, 60, 69, 72, 73, 107, 108, 109, 110, 162, 172, 180, 181, 182, 195, 196, 229, 230, 231, 232, 285]

In [ ]:
"""select = np.arange(48,523).tolist()
select.append(3)
select.append(5)
select.append(6)
select.append(27)"""

In [5]:
df_feat_1 = df_features.loc[:,df_features.columns[select]]
df_feat_1["Image"] = [x.split("_")[0] for x in df_feat_1.FileName_Actin]
df_feat_1.shape

(240, 27)

#### Opening Index csv

In [6]:
df_index = pd.read_csv(index_file, delimiter="\t")

In [ ]:
#df_index

In [ ]:
"""for x,y, in enumerate(df_index.columns):
    print(f"{x}: {y}")"""

In [7]:
select = [36,37,38,43]
df_index_1 = df_index.loc[:,df_index.columns[select]]
df_index_1["Image"] = [x.split(".")[0] for x in df_index_1.sourcefilename]
df_index_1 = df_index_1.drop_duplicates()
df_merged = pd.merge(df_feat_1, df_index_1, how="left", left_on="Image", right_on="Image")
df_merged["Index"] = [x.split("-")[1] for x in df_merged.Image]

In [ ]:
df_merged.shape

In [8]:
save_file = f"{filepath}\output_27feat_1ImagePerWell.csv"
df_merged.to_csv(save_file)

Here quick manual replacement of NaNs, check of / etc.

#### Pre-requisites for functional omero.table

- eine Column "Well" mit der Well ID
- no empty headers
-  ``/`` character is not allowed in object names

In [ ]:
#data_file = f"Z:\Data\@MiN\Sarah\Projects\TiM2023\ExampleData\Results_All\All_output.csv"
#df_features = pd.read_csv(data_file, delimiter=",")

In [9]:
df_features = df_merged.copy()

In [10]:
# Test upload of key:value pairs to wells
conn=ezomero.connect(OMEROUSER, OMEROPASS, "", host=OMEROHOST, port=OMEROPORT, secure=True) 
plate = conn.getObject("Plate", plate_id)
list_wellrow, list_wellcolumn, list_wellid, list_wellpos, list_imageid, list_index = [], [], [], [], [], []

print("\nNumber of fields:", plate.getNumberOfFields())
print("\nGrid size:", plate.getGridSize())
print("\nWells in Plate:", plate.getName())
for well in plate.listChildren():
    index = well.countWellSample()

    for index in range(0, index):
        list_wellid.append(well.getId())
        list_wellpos.append(well.getWellPos())
        list_wellrow.append(well.row)
        list_wellcolumn.append(well.column)
        list_imageid.append(well.getImage(index).getId())
        list_index.append(index+1)


Number of fields: (0, 29)

Grid size: {'rows': 15, 'columns': 23}

Wells in Plate: LM2_GEFGAP_siGENOME_1A


In [14]:
df_wells = pd.DataFrame()
df_wells["Well_ID"] = list_wellid
df_wells["Well_Position"] = list_wellpos
df_wells["Well_Row"] = list_wellrow
df_wells["Well_Column"] = list_wellcolumn
df_wells["Image_ID"] = list_imageid
df_wells["Index"] = list_index
#df_features_plate = df_features[df_features["Plate"] == plate_name]

In [15]:
df_features["Well_Index"] = df_features.WellName + "_" + df_features.Index.astype(str)
df_wells["Well_Index"] =df_wells.Well_Position + "_" + df_wells.Index.astype(str)
df_features_plate = df_features
df_features_plate = df_features_plate.merge(df_wells, left_on="Well_Index", right_on="Well_Index")
df_features_plate["Well"] = df_features_plate.Well_ID
df_features_plate.columns
df_features_plate.drop(["Index_x", "Well_Index", "Well_ID", "Index_y"], axis=1, inplace=True)
df_features_plate.drop(["Row", "Column", "WellName"], axis=1, inplace=True)
df_features_plate.fillna(0, inplace=True)

In [16]:
data_file_to_save = f"{filepath}\OmeroTable_ImageIDs_27feat_1ImagePlate.csv"
df_features_plate.to_csv(data_file_to_save)

In [17]:
from omero.grid import DoubleColumn, ImageColumn, LongColumn, WellColumn, StringColumn, FileColumn
from omero.constants.namespaces import NSBULKANNOTATIONS
from omero.gateway import FileAnnotationWrapper
from omero.model import OriginalFileI

cols = []

for col in df_features_plate.columns:
    #if col == 'Well':
        #cols.append(WellColumn(col, '', df_features_plate[col]))
    if col == "Image_ID":
        cols.append(ImageColumn(col, '', df_features_plate[col]))
    elif df_features_plate[col].dtype == 'int64':
        cols.append(LongColumn(col, '', df_features_plate[col]))
    elif df_features_plate[col].dtype == 'float64':
        cols.append(DoubleColumn(col, '', df_features_plate[col]))

In [20]:
conn=ezomero.connect(OMEROUSER, OMEROPASS, "", host=OMEROHOST, port=OMEROPORT, secure=True) 
screen = conn.getObject("Screen", 852)
resources = conn.c.sf.sharedResources()
repository_id = resources.repositories().descriptions[0].getId().getValue()
table_name = plate_name +"_CellprofilerResults_27feat_1ImagePWell"
table = resources.newTable(repository_id, table_name)
table.initialize(cols)
table.addData(cols)

# Link the table to the plate
orig_file = table.getOriginalFile()
file_ann = FileAnnotationWrapper(conn)
file_ann.setNs(NSBULKANNOTATIONS)
file_ann._obj.file = OriginalFileI(orig_file.id.val, False)
file_ann.save()

#ezomero.post_map_annotation(conn, object_type, object_id, kv_dict, ns, across_groups=True)

screen.linkAnnotation(file_ann)
table.close()

In [19]:
conn.close()